### This Jupyter Notebook generates the dF/F profile along the stimulated dendrites around the stimulated spine, and combines the profile from all the dendrites into a master data file.
#### Input: 
csv files containing fluorescent profiles of individual dendrites with names specifiying experimental specifications separated by "_", generated by Image J macro "plot_profile.ijm". The contain the following columns:  
- X: x coordinates of each point along the polyline, with an increment of 0.2um (or 1 pixel). Positive: distal; negative: proximal.  
- BG: background values for each stimulated point on each frame of each channel   
- Y_pre_Cx: Baseline fluorescent values for each point along the polyline in channel x (0 = CytoRCaMP, 1 = MitoGCaMP or OMMGCaMP)  
- Y-bg_pre_Cx: Y_pre_Cx substracting background  
- Y_post_Cx: Poststim fluorescent values for each point along the polyline in channel x (0 = CytoRCaMP, 1 = MitoGCaMP or OMMGCaMP)  
- Y-bg_post_Cx: Y_post_Cx substracting background  
#### Output:
1. batch_df_name.csv: specifications of each plotted dendrite of each image file. This needs the user to review, add specific column names and fill up missing information in the columns mannually. The reviewed file will be saved as a separate csv file as batch_df_name_copy.csv
2. dF/F and dF profile data for each dendrite saved individually in the output folder. It will contain additional columns compared to the input csv files:  
   - dF/F_Cx: signal fold change upon stimulation calculated for each point along the polyline in channel x (0 = CytoRCaMP, 1 = MitoGCaMP or OMMGCaMP)  
4. batch_profile_summary.csv: combined profile data saved in a master csv file. The rolling average will be calculated using excel on this data file.

## Import packages

In [1]:
import pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline

from math import nan
from typing import ClassVar, Type
from glob import glob
import pprint
import pylab as P
from tqdm import tqdm

In [2]:
import os
from os import listdir
from os.path import isfile, join
cwd = os.getcwd()
cwd

'C:\\Users\\fanr\\OneDrive - Max Planck Florida Institute for Neuroscience\\Codes\\mitoGCaMP\\manuscript'

## Import files and extract file information
Import files and extract information about experimental conditions from image file names and creates inventories of image files and ROIs within each file for later data analysis. This file needs the user to review, add specific column names and fill up missing information in the columns mannually. The reviewed file will be saved as a separate csv file as batch_df_name_copy.csv

In [43]:
#mypath: path for source folder
mypath = ''

#Specify batch name. This will be the prefix of output file names.
batch = 'BATCH NAME'


onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

df_name = []
for i in onlyfiles:
    name = i[0:-4].split("_")
    df_name.append([i] + name[1:])
    
df_name = pd.DataFrame(df_name)

df_name.to_csv(batch+'_df_name.csv',index = False)

## Calculate the dF/F profile and export as individual csv files for each dendrite. Combine all plot profile data into a master dataframe for subsequent analysis.
For each channel, dF/F  are calculated as (F-F0)/F0, where F is "Y-bg_post", i.e. the background-subtracted fluorescent intensity post-stimulation on a given point of the dendrite; F0 is "Y-bg_pre", i.e. the background-subtracted fluorescent intensity pre-stimulation on the same point of the dendrite. 

In [47]:
# Load reviewed and user-editted name file 
df_name = pd.read_csv(batch+'_df_name_copy.csv')
df_name

,File,Batch,Dish,Cell,C1,C0,Stim
0,AVG_041521_Dish1_cell1_488mitoOMMCaMP6f_561RCa...,41521,Dish1,cell1,488OMMCaMP6f,561RCaMP107,1
1,AVG_041521_Dish1_cell3_488mitoCaMP6f_561RCaMP1...,41521,Dish1,cell3,488mitoGCaMP6f,561RCaMP107,1
2,AVG_041521_Dish1_cell3_488mitoOMMCaMP6f_561RCa...,41521,Dish1,cell3,488OMMCaMP6f,561RCaMP107,1
3,AVG_041521_Dish2_cell1_488mitoCaMP6f_561RCaMP1...,41521,Dish2,cell1,488mitoGCaMP6f,561RCaMP107,1
4,AVG_041521_Dish3_cell1_488mitoOMMCaMP6f_561RCa...,41521,Dish3,cell1,488OMMCaMP6f,561RCaMP107,1
5,AVG_051921_Dish1_cell3_488mitoGCaMP6f_561RCaMP...,51921,Dish1,cell3,488mitoGCaMP6f,561RCaMP107,1
6,AVG_051921_Dish5_cell3_488mitoOMMGCaMP_561RCaM...,51921,Dish5,cell3,488OMMCaMP6f,561RCaMP107,1
7,AVG_052621_Dish4_cell2_488mitoGCaMP6f_561RCaMP...,52621,Dish4,cell2,488mitoGCaMP6f,561RCaMP107,1
8,AVG_052621_Dish4_cell2_488mitoGCaMP6f_561RCaMP...,52621,Dish4,cell2,488mitoGCaMP6f,561RCaMP107,2
9,AVG_052621_Dish6_cell2_488mitoOMMGCaMP6f_561RC...,52621,Dish6,cell2,488OMMCaMP6f,561RCaMP107,1


In [49]:
#output: path for output folders
output = ''

In [55]:
df_summary=[]
ncol=0
ncol_bin=0
df_all= pd.DataFrame()
df_all2= pd.DataFrame()
for i in onlyfiles:
    df_name_i = df_name.loc[df_name.File == i]
    
    df = pd.read_csv(mypath+'\\'+i)
    df = df.iloc[:,2:]
    channel = []
    for c in df.columns[1:]:
        channel = channel + [c.split("_")[-1]]
    channel = list(set(channel))

    for ch in channel:
        df['dF/F_'+ch] = (df['Y-bg_post_'+ch]-df['Y-bg_pre_'+ch])/abs(df['Y-bg_pre_'+ch])

    df=df.sort_values(by='X').reset_index(drop=True)
    df = df.groupby('X').mean().reset_index()
    df.to_csv(output+'\\'+i,index=False)
    

    df['C0']=df_name_i['C0'].tolist()*df.shape[0]
    df['C1']=df_name_i['C1'].tolist()*df.shape[0]
    
    
    l = 40 # Specify the length of the dendrite to be kept on each side of the stimulated spine.
           # The data point will be removed if the profile plotted is not longer than l on both sides of the stimulated spine.  
    
    col_sel=[]
  
    if (df['X'].max()>=l)&(df['X'].min()<=-l):
        df = df[(df['X']<=l) & (df['X']>=-l)]
        
        col_sel = [col for col in df.columns if 'dF/F' in col]
        df_all = pd.concat([df_all,df[['C0','C1','X']+col_sel]])

        
        ncol=max(ncol,len(df['X'].values.tolist()))
        
        arr_x = df_name_i.values.tolist()[0] +['X']+ df['X'].values.tolist()
        df_summary = df_summary+[arr_x]
        for ch in channel:
            arr = df_name_i.values.tolist()[0] +['dF/F_'+ch] +df['dF/F_'+ch].values.tolist()
            df_summary = df_summary+[arr]

      
df_summary_40= pd.DataFrame(df_summary,columns =df_name.columns.to_list()+['Data']+ list(range(ncol)))
df_summary_40

,File,Batch,Dish,Cell,C1,C0,Stim,Data,0,1,...,389,390,391,392,393,394,395,396,397,398
0,AVG_041521_Dish1_cell1_488mitoOMMCaMP6f_561RCa...,41521,Dish1,cell1,488OMMCaMP6f,561RCaMP107,1,X,-39.867000,-39.667000,...,38.064000,38.265000,38.465000,38.665000,38.866000,39.066000,39.266000,39.467000,39.667000,39.867000
1,AVG_041521_Dish1_cell1_488mitoOMMCaMP6f_561RCa...,41521,Dish1,cell1,488OMMCaMP6f,561RCaMP107,1,dF/F_C0,0.072825,-0.048187,...,-0.212272,-0.234446,0.182464,0.057160,0.067721,-0.148111,-0.281396,-0.005007,0.148273,0.048329
2,AVG_041521_Dish1_cell1_488mitoOMMCaMP6f_561RCa...,41521,Dish1,cell1,488OMMCaMP6f,561RCaMP107,1,dF/F_C1,-0.606610,-0.373295,...,-0.731446,-0.536950,-0.528636,2.059604,0.132575,-0.096435,-0.095596,-0.250213,-0.282089,-0.193688
3,AVG_041521_Dish1_cell3_488mitoCaMP6f_561RCaMP1...,41521,Dish1,cell3,488mitoGCaMP6f,561RCaMP107,1,X,-39.867000,-39.667000,...,38.064000,38.265000,38.465000,38.665000,38.866000,39.066000,39.266000,39.467000,39.667000,39.867000
4,AVG_041521_Dish1_cell3_488mitoCaMP6f_561RCaMP1...,41521,Dish1,cell3,488mitoGCaMP6f,561RCaMP107,1,dF/F_C0,-0.078063,-0.097477,...,-0.129594,-0.082503,-0.105580,0.040459,-0.078766,-0.130280,-0.103013,-0.031415,-0.046234,-0.063201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,AVG_081721_Dish2_Cell8-1_488mitoGCaMP6f_561RCa...,81721,Dish2,Cell8-1,488mitoGCaMP6f,561RCaMP107,1,dF/F_C0,0.016705,0.012964,...,-0.000758,0.007252,-0.010319,-0.005567,-0.034777,-0.001637,0.009303,-0.009540,-0.069580,-0.010897
86,AVG_081721_Dish2_Cell8-1_488mitoGCaMP6f_561RCa...,81721,Dish2,Cell8-1,488mitoGCaMP6f,561RCaMP107,1,dF/F_C1,0.071734,-0.122265,...,-0.232284,-0.463042,-0.470292,0.276822,0.890970,-0.153537,-0.125410,-0.356345,0.005446,0.358319
87,AVG_081721_Dish2_Cell8-1_488mitoGCaMP6f_561RCa...,81721,Dish2,Cell8-1,488mitoGCaMP6f,561RCaMP107,2,X,-39.867000,-39.667000,...,38.064000,38.265000,38.465000,38.665000,38.866000,39.066000,39.266000,39.467000,39.667000,39.867000
88,AVG_081721_Dish2_Cell8-1_488mitoGCaMP6f_561RCa...,81721,Dish2,Cell8-1,488mitoGCaMP6f,561RCaMP107,2,dF/F_C0,-0.024052,-0.039308,...,-0.037489,-0.032621,-0.011730,0.026798,0.051790,-0.000434,-0.037821,-0.037438,-0.055888,-0.047230


In [53]:
# Export master csv files. Rolling average was calculated in excel.
df_summary_40.to_csv(batch+'_profile_summary.csv',index = False)